In [4]:
import utils
import json
import sys
sys.path.append('/home/jupyter/tldr-reddit-summarization/pegasus')
import pegasus
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
tf.__version__

# Create TFRecords for PEGASUS library

In [5]:
reddit_posts = utils.load_reddit_data_from_json()

0
1000000
2000000
3000000


In [7]:
subreddits = []
content = []
summary = []

while reddit_posts:
    post = reddit_posts.pop()
    if 'subreddit' in post:
        subreddits.append(post['subreddit'])
        content.append(post['content'])
        summary.append(post['summary'])

In [20]:
subreddits = tf.convert_to_tensor(subreddits)
content = tf.convert_to_tensor(content)
summary = tf.convert_to_tensor(summary)

In [22]:
features_dataset = tf.data.Dataset.from_tensor_slices((subreddits, content, summary))

In [24]:
# https://www.tensorflow.org/tutorials/load_data/tfrecord
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(subreddit, content, summary):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
      'subreddits': _bytes_feature(subreddit),
      'inputs':  _bytes_feature(content),
      'targets': _bytes_feature(summary),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [25]:
def tf_serialize_example(subreddit, content, summary):
    tf_string = tf.py_function(
        serialize_example,
        (subreddit, content, summary),  # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar

In [27]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)

# def generator():
#     for features in features_dataset:
#         yield serialize_example(*features)

# serialized_features_dataset = tf.data.Dataset.from_generator(
#     generator, output_types=tf.string, output_shapes=())

In [28]:
filename = 'Reddit.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

# Run PEGASUS finetuning